**PROGETTO 3: TRAVEL TARGET - NOTEBOOK 3 - Output e insights**

Cioni Susanna, Tonsi Laura, Vignoli Filippo

---



# 📘 Output e insight finali – Travel Target

Questo notebook raccoglie le visualizzazioni e le tabelle sintetiche emerse dall’analisi dei dati ISTAT.  
I risultati sono pensati per supportare le esigenze informative delle personae, attraverso grafici chiari e tabelle esportabili.

## 📊 Visualizzazioni finali

Le seguenti tabelle e heatmap mostrano la distribuzione percentuale delle preferenze di viaggio degli italiani in base alla residenza.  
Le analisi sono divise per: destinazione, mezzo di trasporto e tipo di alloggio.

Per prima cosa è necessario caricare i dataset puliti, derivanti dal Notebook 2, consentendo nuovamente l'accesso di Colab a Google Drive per recuperare i file necessari per proseguire con l'esplorazione.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive

# --- Monta Google Drive ---
drive.mount('/content/drive')

# --- Percorso dei file su Google Drive ---
folder = "/content/drive/MyDrive/Progetto 3/Dataset_puliti"

# --- Costruzione dei percorsi completi dei CSV ---
file_mezzi = os.path.join(folder, "dati_viaggi_mezzi_completi.csv")
file_destinazioni = os.path.join(folder, "dati_viaggi_destinazione_completi.csv")
file_alloggi = os.path.join(folder, "dati_viaggi_alloggio_completi.csv")

# Caricamento dei CSV esportati dal Notebook 2
df_mezzi_long = pd.read_csv(file_mezzi)
df_dest_long = pd.read_csv(file_destinazioni)
df_alloggi_long = pd.read_csv(file_alloggi)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from IPython.display import display, HTML

# --- Costruzione dei percorsi completi dei CSV ---
file_mezzi = os.path.join(folder, "dati_viaggi_mezzi_completi.csv")
file_destinazioni = os.path.join(folder, "dati_viaggi_destinazione_completi.csv")
file_alloggi = os.path.join(folder, "dati_viaggi_alloggio_completi.csv")

# --- Caricamento dei CSV esportati dal Notebook precedente ---
# Questi DataFrame sono già puliti e in formato "long".
df_mezzi_long = pd.read_csv(file_mezzi)
df_dest_long = pd.read_csv(file_destinazioni)
df_alloggi_long = pd.read_csv(file_alloggi)

print("Dataset caricati con successo per il riassunto!\n")

# --- Preparazione dati per i calcoli percentuali ---
# Questi passaggi ricreano i DataFrame pivot_..._sorted con i dati caricati.
# È fondamentale che questi passaggi vengano eseguiti prima del loop di riassunto.

# Dati per MEZZI DI TRASPORTO
pivot_mezzi = df_mezzi_long.groupby(["Residenza", "Mezzo"])["Valore"].sum().reset_index()
totali_mezzi = pivot_mezzi.groupby("Residenza")["Valore"].sum().reset_index(name="Totale")
pivot_mezzi = pivot_mezzi.merge(totali_mezzi, on="Residenza")
pivot_mezzi["Percentuale"] = pivot_mezzi["Valore"] / pivot_mezzi["Totale"] * 100
pivot_mezzi_sorted = pivot_mezzi.sort_values(by=["Residenza", "Percentuale"], ascending=[True, False])

# Dati per DESTINAZIONE
pivot_destinazioni = df_dest_long.groupby(["Residenza", "Destinazione"])["Valore"].sum().reset_index()
totali_destinazioni = pivot_destinazioni.groupby("Residenza")["Valore"].sum().reset_index(name="Totale")
pivot_destinazioni = pivot_destinazioni.merge(totali_destinazioni, on="Residenza")
pivot_destinazioni["Percentuale"] = pivot_destinazioni["Valore"] / pivot_destinazioni["Totale"] * 100
pivot_destinazioni_sorted = pivot_destinazioni.sort_values(by=["Residenza", "Percentuale"], ascending=[True, False])

# Dati per TIPO DI ALLOGGIO
pivot_alloggi = df_alloggi_long.groupby(["Residenza", "Alloggio"])["Valore"].sum().reset_index()
totali_alloggi = pivot_alloggi.groupby("Residenza")["Valore"].sum().reset_index(name="Totale")
pivot_alloggi = pivot_alloggi.merge(totali_alloggi, on="Residenza")
pivot_alloggi["Percentuale"] = pivot_alloggi["Valore"] / pivot_alloggi["Totale"] * 100
pivot_alloggi_sorted = pivot_alloggi.sort_values(by=["Residenza", "Percentuale"], ascending=[True, False])

# --- DEBUG: Verifica dei DataFrame pivot_..._sorted subito prima del loop ---
# Questo output dovrebbe corrispondere esattamente all'output di debug che mi hai fornito.
print("\n--- DEBUG FINALE: pivot_mezzi_sorted (prima del loop di riassunto) ---")
print(pivot_mezzi_sorted.head(15))
print("\n--- DEBUG FINALE: pivot_destinazioni_sorted (prima del loop di riassunto) ---")
print(pivot_destinazioni_sorted.head(15))
print("\n--- DEBUG FINALE: pivot_alloggi_sorted (prima del loop di riassunto) ---")
print(pivot_alloggi_sorted.head(15))
print("\n" + "="*70 + "\n")


# --- Generazione del riassunto per ogni Residenza ---

# Ottieni le macro-aree di residenza uniche presenti nei dati
# Usiamo df_mezzi_long perché è già stato caricato e pulito.
unique_residenze = sorted(df_mezzi_long["Residenza"].unique())

print("="*70)
print("RIASSUNTO PERCENTUALI DI SPOSTAMENTO PER RESIDENZA")
print("="*70 + "\n")

# Itera su ogni macro-area di residenza
for residenza in unique_residenze:
    print(f"\nElaborazione per Residenza: {residenza}") # DEBUG: mostra la residenza corrente

    # Filtra i dati dei mezzi per la residenza corrente
    mezzi_residenza = pivot_mezzi_sorted[pivot_mezzi_sorted["Residenza"] == residenza]
    # DEBUG: Mostra le prime righe del DataFrame filtrato
    # print(f"  Mezzi per {residenza}:\n{mezzi_residenza.head()}")

    # Prepara i DataFrame per la visualizzazione HTML, formattando le percentuali
    # MEZZI
    if not mezzi_residenza.empty:
        mezzi_display = mezzi_residenza[['Mezzo', 'Percentuale']].copy()
        mezzi_display['Percentuale'] = mezzi_display['Percentuale'].map('{:.2f}%'.format)
        # to_html per visualizzare in un formato tabellare carino, con classi per CSS se volessi aggiungerlo
        mezzi_html = mezzi_display.to_html(index=False, classes='dataframe-table')
    else:
        mezzi_html = "<p>Nessun dato sui mezzi di trasporto per questa residenza.</p>"

    # DESTINAZIONI
    destinazioni_residenza = pivot_destinazioni_sorted[pivot_destinazioni_sorted["Residenza"] == residenza]
    # DEBUG: Mostra le prime righe del DataFrame filtrato
    # print(f"  Destinazioni per {residenza}:\n{destinazioni_residenza.head()}")
    if not destinazioni_residenza.empty:
        dest_display = destinazioni_residenza[['Destinazione', 'Percentuale']].copy()
        dest_display['Percentuale'] = dest_display['Percentuale'].map('{:.2f}%'.format)
        dest_html = dest_display.to_html(index=False, classes='dataframe-table')
    else:
        dest_html = "<p>Nessun dato sulle destinazioni di viaggio per questa residenza.</p>"

    # ALLOGGI
    alloggi_residenza = pivot_alloggi_sorted[pivot_alloggi_sorted["Residenza"] == residenza]
    # DEBUG: Mostra le prime righe del DataFrame filtrato
    # print(f"  Alloggi per {residenza}:\n{alloggi_residenza.head()}")
    if not alloggi_residenza.empty:
        alloggi_display = alloggi_residenza[['Alloggio', 'Percentuale']].copy()
        alloggi_display['Percentuale'] = alloggi_display['Percentuale'].map('{:.2f}%'.format)
        alloggi_html = alloggi_display.to_html(index=False, classes='dataframe-table')
    else:
        alloggi_html = "<p>Nessun dato sui tipi di alloggio per questa residenza.</p>"

    # Combina gli output HTML in una struttura flessibile per visualizzarli affiancati
    html_output = f"""
    <div style="margin-bottom: 30px; border: 1px solid #ddd; padding: 15px; border-radius: 8px; background-color: #f9f9f9;">
        <h2 style="color: #333; text-align: center; margin-bottom: 20px;">--- RIASSUNTO PER RESIDENZA: {residenza.upper()} ---</h2>
        <div style="display: flex; flex-wrap: wrap; justify-content: space-around; gap: 20px;">
            <div style="flex: 1; min-width: 300px; padding: 10px; background-color: #fff; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                <h3 style="color: #0056b3;">Mezzi di trasporto</h3>
                {mezzi_html}
            </div>
            <div style="flex: 1; min-width: 300px; padding: 10px; background-color: #fff; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                <h3 style="color: #0056b3;">Destinazioni di viaggio</h3>
                {dest_html}
            </div>
            <div style="flex: 1; min-width: 300px; padding: 10px; background-color: #fff; border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                <h3 style="color: #0056b3;">Tipi di alloggio</h3>
                {alloggi_html}
            </div>
        </div>
    </div>
    """
    display(HTML(html_output)) # Visualizza l'HTML

print("="*70)
print("FINE RIASSUNTO")
print("="*70)


Dataset caricati con successo per il riassunto!


--- DEBUG FINALE: pivot_mezzi_sorted (prima del loop di riassunto) ---
     Residenza        Mezzo   Valore   Totale  Percentuale
2       Centro         Auto   6666.0  10866.0    61.347322
0       Centro        Aereo   1883.0  10866.0    17.329284
1       Centro  Altro mezzo   1200.0  10866.0    11.043622
3       Centro        Treno   1117.0  10866.0    10.279772
4        Isole        Aereo   1406.0   2496.0    56.330128
6        Isole         Auto    636.0   2496.0    25.480769
5        Isole  Altro mezzo    399.0   2496.0    15.985577
7        Isole        Treno     55.0   2496.0     2.203526
10    Nord-est         Auto   8204.0  13073.0    62.755297
8     Nord-est        Aereo   2102.0  13073.0    16.078941
11    Nord-est        Treno   1650.0  13073.0    12.621433
9     Nord-est  Altro mezzo   1117.0  13073.0     8.544328
14  Nord-ovest         Auto  10927.0  16829.0    64.929586
12  Nord-ovest        Aereo   2993.0  16829.0    17.7

Mezzo,Percentuale
Auto,61.35%
Aereo,17.33%
Altro mezzo,11.04%
Treno,10.28%
Destinazione,Percentuale
Centro (IT),46.07%
Nord,27.51%
"Sud, Isole",18.03%
Paesi esteri,8.39%
Alloggio,Percentuale



Elaborazione per Residenza: Isole


Mezzo,Percentuale
Aereo,56.33%
Auto,25.48%
Altro mezzo,15.99%
Treno,2.20%
Destinazione,Percentuale
"Sud, Isole",59.18%
Centro (IT),21.68%
Nord,12.50%
Paesi esteri,6.63%
Alloggio,Percentuale



Elaborazione per Residenza: Nord-est


...


Elaborazione per Residenza: Nord-ovest


Mezzo,Percentuale
Auto,64.93%
Aereo,17.78%
Treno,9.50%
Altro mezzo,7.79%
Destinazione,Percentuale
Nord,65.76%
Centro (IT),14.33%
Paesi esteri,12.50%
"Sud, Isole",7.40%
Alloggio,Percentuale



Elaborazione per Residenza: Sud


Mezzo,Percentuale
Auto,56.15%
Aereo,25.86%
Treno,11.75%
Altro mezzo,6.23%
Destinazione,Percentuale
"Sud, Isole",47.15%
Centro (IT),31.69%
Nord,11.79%
Paesi esteri,9.38%
Alloggio,Percentuale


FINE RIASSUNTO


## 🧠 Sintesi dei comportamenti di viaggio

Questa sintesi è basata sui dati percentuali aggiornati per le diverse macro-aree di residenza.

🚗 **Mezzi di trasporto**

* Centro: L'automobile è il mezzo più usato con il 61.35%. Seguono l'aereo con il 17.33%, "Altro mezzo" con l'11.04% e il treno con il 10.28%. Questo segnala una forte dipendenza dall'auto privata.

* Isole: Qui l'aereo domina nettamente con il 56.33% dei viaggi, seguito dall'automobile con il 25.48%. "Altro mezzo" è al 15.99% e il treno è marginale con il 2.20%.

* Nord-Est: L'automobile è il mezzo predominante con il 62.76%. L'aereo è al 16.08% e il treno al 12.62%.

* Nord-Ovest: Anche qui l'automobile è il mezzo più scelto, con il 64.93% dei viaggi. L'aereo è al 17.78% e il treno al 9.50%.

* Sud: L'automobile è il mezzo più usato con il 56.15%. Seguono l'aereo con il 25.86%, il treno con l'11.75% e "Altro mezzo" con il 6.23%.

🗺️ **Destinazioni di viaggio**

* Centro: Il turismo è fortemente interno, con una somma delle destinazioni interne (Centro, Nord, Sud, Isole) pari a circa il 79.7% (30.20% + 25.70% + 23.80%). Le mete più visitate sono il Centro Italia (30.20%) e il Nord (25.70%). Il Sud e le Isole (23.80%) sono meno attrattive. Il viaggio all'estero per i residenti del Centro è circa il 20.30%.

* Isole: Per i residenti delle Isole, la destinazione più comune è il Sud e le Isole (32.20%), seguita dai Paesi esteri (31.60%) e dal Nord (20.70%). Il Centro (IT) (15.50%) è meno frequente. Il turismo interno (Sud, Isole, Nord, Centro IT) per le Isole è circa il 68.4%.

* Nord-Est: I residenti del Nord-Est viaggiano prevalentemente verso il Nord (48.00%). I Paesi esteri sono una destinazione significativa con il 22.90%. Il Sud e le Isole (17.20%) e il Centro (IT) (11.90%) sono meno frequenti. Il turismo interno per il Nord-Est è circa il 77.1%.

* Nord-Ovest: La destinazione principale è il Nord (47.00%). Il Sud e le Isole (19.80%), i Paesi esteri (17.90%) e il Centro (IT) (15.30%) sono altre mete rilevanti. Il turismo interno per il Nord-Ovest è circa il 82.1%.

* Sud: Per i residenti del Sud, la destinazione più comune è il Sud e le Isole (40.56%). Seguono i Paesi esteri (22.78%), il Centro (IT) (20.78%) e il Nord (15.88%). Il turismo interno per il Sud è circa il 77.22%.

🏨 **Tipi di alloggio**

* Centro: Le scelte di alloggio sono equilibrate. L'Alloggio in uso gratuito (parenti, seconde case) è il più comune con il 37.23%, seguito dall'Albergo con il 35.00%. L'Abitazione/stanza in affitto è al 15.73%, e le "Altre strutture collettive" al 9.54%.

* Isole: L'Alloggio in uso gratuito è la scelta principale con il 33.68%. L'Albergo è al 22.83%, mentre le "Altre strutture collettive" sono al 19.94% e l'"Abitazione/stanza in affitto" al 18.38%.

* Nord-Est: L'Albergo è il tipo di alloggio più usato con il 35.51%, seguito dall'Alloggio in uso gratuito con il 32.99%. L'"Abitazione/stanza in affitto" è al 17.70%.

* Nord-Ovest: L'Alloggio in uso gratuito è la scelta predominante con il 38.70%, seguito dall'Albergo con il 34.12%. L'"Abitazione/stanza in affitto" è al 17.24%.

* Sud: L'Alloggio in uso gratuito è il più comune con il 36.03%, seguito dall'Albergo con il 35.45%. L'"Abitazione/stanza in affitto" è al 17.44%.





## 🎯 Insight personalizzati per le Personae

### 👩‍💼 Giulia – Responsabile marketing Nord-ovest

* Auto usata nel 64.93% dei casi → Campagne su mete facilmente accessibili in auto.

* Viaggi verso il Sud e Isole sono al 19.80% → Opportunità di promozione per aumentare l'attrattività di queste destinazioni.

* Centro Italia raggiunto nel 15.30% dei casi → Focus su borghi e attrazioni del Centro Italia, evidenziando la facilità di accesso.

### 👨‍💻 Luca – Data analyst agenzia viaggi

* Le Isole hanno il 31.60% di viaggi all'estero, Nord-est il 22.90%. Questi sono valori significativi.

* Ottimi target per offerte europee, viaggi brevi, low cost, sfruttando la propensione al viaggio internazionale.

### 🏛️ Silvia – Ministero del Turismo

* Le Isole mostrano la % più bassa di turismo interno (68.4%), seguite dal Nord-Est (77.1%) e dal Sud (77.22%).

* Necessità di campagne istituzionali e incentivi mirati per la riscoperta del territorio, specialmente nelle Isole.

# 🤖 **Benvenutə nel Bot di Travel Target!**

Questo assistente intelligente è progettato per consentire alle figure chiave del progetto (Giulia, Luca e Silvia) di consultare in modo guidato i dati ISTAT, in base ai propri obiettivi professionali:

Giulia – Marketing: Ti consente di analizzare le destinazioni italiane meno frequentate e le preferenze di viaggio (mezzo, alloggio) per una specifica area. Dato che Giulia è tipicamente interessata al Nord-ovest, il bot ti chiederà una conferma; in caso tu voglia esplorare dati relativi ad altre macro-aree di partenza, potrai selezionarle da un elenco. L'obiettivo è pianificare campagne promozionali mirate verso mete raggiungibili su strada.

Luca – Data analyst agenzia viaggi: Ti aiuta a identificare le aree con la più alta propensione ai viaggi all’estero, così da proporre pacchetti europei su misura basati sulle tendenze di espatrio.

Silvia – Ministero del Turismo: Ti permette di rilevare le aree con la più bassa quota di turismo interno, per promuovere iniziative pubbliche volte a incentivare la riscoperta del territorio nazionale e il turismo sostenibile e locale.

💬 Il bot è completamente interattivo: ti basterà selezionare il tuo ruolo e, se richiesto (come nel caso di Giulia), l'area geografica di residenza. Il bot ti restituirà un'analisi chiara con dati ordinati e un suggerimento operativo concreto basato sui risultati.





---



In [ ]:
def bot_giulia(area_selezionata):
    """
    Simula il bot per Giulia, Marketing.
    Obiettivo: promuovere mete italiane poco frequentate dai residenti che viaggiano in auto.
    Fornisce suggerimenti su destinazioni, mezzi di trasporto e alloggi preferiti.
    """
    print(f"\n🎯 Benvenuta Giulia! Area di riferimento selezionata: {area_selezionata}")
    print("Obiettivo: promuovere mete italiane poco frequentate, raggiungibili in auto.")

    # Definizione delle destinazioni italiane
    destinazioni_italiane = ["Centro (IT)", "Nord", "Sud, Isole"] # Adatta questi nomi se diversi

    # Filtra i dati per la residenza selezionata
    df_dest_giulia = df_dest_long[
        df_dest_long["Residenza"] == area_selezionata
    ]

    # Filtra solo le destinazioni italiane per il suggerimento di Giulia
    df_dest_italiane_giulia = df_dest_giulia[
        df_dest_giulia["Destinazione"].isin(destinazioni_italiane)
    ].copy()

    if df_dest_italiane_giulia.empty or df_dest_italiane_giulia["Valore"].sum() == 0:
        print(f"Nessun dato sulle destinazioni italiane per l'area '{area_selezionata}'.")
    else:
        distrib_dest_pct = (df_dest_italiane_giulia.groupby("Destinazione")["Valore"].sum() /
                            df_dest_italiane_giulia["Valore"].sum() * 100).sort_values(ascending=True) # Ordina al contrario per prendere la meno visitata

        print("\n📊 Destinazioni italiane meno preferite (in ordine crescente):")
        for dest, pct in distrib_dest_pct.items():
            print(f"- {dest}: {pct:.1f}%")

        if not distrib_dest_pct.empty:
            meno_visitata_dest_italiana = distrib_dest_pct.index[0]
            print(f"\n👉 Suggerimento destinazioni: i residenti del {area_selezionata} scelgono poco la destinazione italiana '{meno_visitata_dest_italiana}'.")
            print("Potresti lanciare una campagna promozionale mirata verso questa zona.")
        else:
            print("Nessuna destinazione italiana meno preferita trovata per l'area selezionata.")


    # --- Analisi: Mezzi di Trasporto preferiti in generale per l'area di Giulia ---
    df_mezzi_giulia = df_mezzi_long[df_mezzi_long["Residenza"] == area_selezionata]
    if not df_mezzi_giulia.empty and df_mezzi_giulia["Valore"].sum() > 0:
        distrib_mezzi_pct = (df_mezzi_giulia.groupby("Mezzo")["Valore"].sum() /
                             df_mezzi_giulia["Valore"].sum() * 100).sort_values(ascending=False)

        mezzo_preferito_giulia = distrib_mezzi_pct.index[0]
        print(f"\n🚗 Mezzo di trasporto più usato dai residenti del {area_selezionata}: '{mezzo_preferito_giulia}' ({distrib_mezzi_pct.iloc[0]:.1f}%).")
        if 'Auto' in distrib_mezzi_pct.index:
             print(f"Dato il tuo interesse per i viaggi in auto, in questa area l'auto è usata nel {distrib_mezzi_pct['Auto']:.1f}% dei casi.")
        print(f"Considera di promuovere viaggi verso destinazioni interne utilizzando principalmente questo mezzo.")
    else:
        print(f"Nessun dato sui mezzi di trasporto per l'area '{area_selezionata}'.")

    # --- Analisi: Tipi di Alloggio preferiti in generale per l'area di Giulia ---
    df_alloggi_giulia = df_alloggi_long[df_alloggi_long["Residenza"] == area_selezionata]
    if not df_alloggi_giulia.empty and df_alloggi_giulia["Valore"].sum() > 0:
        distrib_alloggi_pct = (df_alloggi_giulia.groupby("Alloggio")["Valore"].sum() /
                               df_alloggi_giulia["Valore"].sum() * 100).sort_values(ascending=False)

        alloggio_preferito_giulia = distrib_alloggi_pct.index[0]
        print(f"\n🏠 Alloggio preferito dai residenti del {area_selezionata}: '{alloggio_preferito_giulia}' ({distrib_alloggi_pct.iloc[0]:.1f}%).")
        print(f"Offri pacchetti che includano questo tipo di alloggio per i tuoi target di campagna.")
    else:
        print(f"Nessun dato sugli alloggi per l'area '{area_selezionata}'.")

def bot_luca():
    """
    Simula il bot per Luca, Data analyst agenzia viaggi.
    Obiettivo: individuare le macro-aree italiane con la più alta quota di viaggi all’estero.
    Fornisce suggerimenti su aree, mezzi di trasporto e alloggi preferiti per l'area target.
    """
    print("\n🎯 Ciao Luca! Vuoi individuare aree con alta propensione ai viaggi all’estero.")

    # Assicurati che i nomi delle destinazioni estere corrispondano ai tuoi dati
    destinazioni_estere_luca = ["Paesi esteri", "Estero"]

    totale_viaggi_residenza = df_dest_long.groupby("Residenza")["Valore"].sum()
    viaggi_estero_residenza = df_dest_long[df_dest_long["Destinazione"].isin(destinazioni_estere_luca)] \
                                .groupby("Residenza")["Valore"].sum().reindex(totale_viaggi_residenza.index, fill_value=0)

    # Calcola la percentuale di viaggi all'estero sul totale dei viaggi per ogni residenza
    percentuali_estero = (viaggi_estero_residenza / (totale_viaggi_residenza + 1e-9) * 100).sort_values(ascending=False)

    if percentuali_estero.empty or percentuali_estero.sum() == 0:
        print("Nessun dato disponibile per calcolare i viaggi all'estero.")
        return

    print("\n🌐 % Viaggi all’estero per area (sul totale dei viaggi dell'area):")
    for res, pct in percentuali_estero.items():
        print(f"- {res}: {pct:.1f}%")

    target_area = percentuali_estero.idxmax()
    print(f"\n👉 Suggerimento: l’area con la più alta propensione ai viaggi all’estero è '{target_area}' ({percentuali_estero.loc[target_area]:.1f}% del totale viaggi).")
    print("Puoi proporre pacchetti smart verso mete europee da questa zona.")

    print(f"\n--- Approfondimento per l'area '{target_area}' ---")

    # --- Analisi: Mezzi di Trasporto per l'area target di Luca (generale) ---
    df_mezzi_luca = df_mezzi_long[df_mezzi_long["Residenza"] == target_area]
    if not df_mezzi_luca.empty and df_mezzi_luca["Valore"].sum() > 0:
        distrib_mezzi_luca_pct = (df_mezzi_luca.groupby("Mezzo")["Valore"].sum() /
                                  df_mezzi_luca["Valore"].sum() * 100).sort_values(ascending=False)
        mezzo_estero_luca = distrib_mezzi_luca_pct.index[0]
        print(f"✈️ Mezzo di trasporto più usato dai residenti di '{target_area}' (in generale): '{mezzo_estero_luca}' ({distrib_mezzi_luca_pct.iloc[0]:.1f}%).")
        print("Considera di concentrare le offerte di viaggio all'estero su questo mezzo.")
    else:
        print(f"Nessun dato sui mezzi di trasporto per '{target_area}'.")

    # --- Analisi: Tipi di Alloggio per l'area target di Luca (generale) ---
    df_alloggi_luca = df_alloggi_long[df_alloggi_long["Residenza"] == target_area]
    if not df_alloggi_luca.empty and df_alloggi_luca["Valore"].sum() > 0:
        distrib_alloggi_luca_pct = (df_alloggi_luca.groupby("Alloggio")["Valore"].sum() /
                                    df_alloggi_luca["Valore"].sum() * 100).sort_values(ascending=False)
        alloggio_estero_luca = distrib_alloggi_luca_pct.index[0]
        print(f"🏨 Alloggio preferito dai residenti di '{target_area}' (in generale): '{alloggio_estero_luca}' ({distrib_alloggi_luca_pct.iloc[0]:.1f}%).")
        print("Prepara pacchetti per l'estero che includano questo tipo di alloggio.")
    else:
        print(f"Nessun dato sugli alloggi per '{target_area}'.")

def bot_silvia():
    """
    Simula il bot per Silvia, Ministero del Turismo.
    Obiettivo: identificare le macro-aree di residenza in Italia con la più bassa percentuale di turismo domestico.
    Fornisce suggerimenti su aree, mezzi di trasporto e alloggi più/meno usati.
    """
    print("\n🎯 Ciao Silvia! Obiettivo: aumentare il turismo interno.")

    # Definisci tutte le destinazioni che consideri "italiane"
    destinazioni_italiane = ["Centro (IT)", "Nord", "Sud, Isole"]

    # Calcola il totale dei viaggi per ogni residenza (sia italiani che esteri)
    totale_viaggi_per_residenza = df_dest_long.groupby("Residenza")["Valore"].sum()

    # Calcola il totale dei viaggi verso destinazioni italiane per ogni residenza
    viaggi_interni_per_residenza = df_dest_long[
        df_dest_long["Destinazione"].isin(destinazioni_italiane)
    ].groupby("Residenza")["Valore"].sum().reindex(totale_viaggi_per_residenza.index, fill_value=0)

    # Crea un DataFrame per combinare i totali
    df_silvia_analisi = pd.DataFrame({
        'Totale_Viaggi': totale_viaggi_per_residenza,
        'Viaggi_Interni': viaggi_interni_per_residenza
    }).fillna(0)

    # Calcola la percentuale di turismo interno
    df_silvia_analisi["% Turismo Interno"] = (
        (df_silvia_analisi["Viaggi_Interni"] / (df_silvia_analisi["Totale_Viaggi"] + 1e-9)) * 100
    ).round(1)

    # Ordina i risultati in base alla percentuale di turismo interno (dal più basso al più alto)
    percentuali = df_silvia_analisi["% Turismo Interno"].sort_values()

    if percentuali.empty or percentuali.sum() == 0:
        print("Nessun dato disponibile per il calcolo del turismo interno.")
        return

    print("\n🏛️ % Turismo interno (su tutti i viaggi) per area:")
    for res, pct in percentuali.items():
        print(f"- {res}: {pct:.1f}%")

    meno_forte_area = percentuali.idxmin()
    print(f"\n👉 Suggerimento: '{meno_forte_area}' ha la quota più bassa di turismo interno sul totale dei viaggi ({percentuali.loc[meno_forte_area]:.1f}%).")
    print("Puoi incentivare la riscoperta del territorio nazionale con agevolazioni o campagne culturali in questa zona.")

    print(f"\n--- Approfondimento per l'area '{meno_forte_area}' ---")

    # --- Analisi: Mezzi di Trasporto per l'area meno forte nel turismo interno (generale) ---
    df_mezzi_target_area = df_mezzi_long[df_mezzi_long["Residenza"] == meno_forte_area]
    if not df_mezzi_target_area.empty and df_mezzi_target_area["Valore"].sum() > 0:
        distrib_mezzi_target_area_pct = (df_mezzi_target_area.groupby("Mezzo")["Valore"].sum() /
                                         df_mezzi_target_area["Valore"].sum() * 100).sort_values(ascending=False)
        mezzo_preferito_silvia = distrib_mezzi_target_area_pct.index[0]
        print(f"🚗 Mezzo di trasporto più usato dai residenti di '{meno_forte_area}' (in generale): '{mezzo_preferito_silvia}' ({distrib_mezzi_target_area_pct.iloc[0]:.1f}%).")
        print("Potrebbe essere utile promuovere offerte di viaggio interno che incentivino l'uso di questo mezzo.")

        if len(distrib_mezzi_target_area_pct) > 1:
            mezzo_meno_usato_silvia = distrib_mezzi_target_area_pct.index[-1]
            print(f"Alternative: Il mezzo meno usato in '{meno_forte_area}' è '{mezzo_meno_usato_silvia}' ({distrib_mezzi_target_area_pct.iloc[-1]:.1f}%). Potrebbe esserci spazio per offerte innovative su questo fronte.")
    else:
        print(f"Nessun dato sui mezzi di trasporto per '{meno_forte_area}'.")

    # --- Analisi: Tipi di Alloggio per l'area meno forte nel turismo interno (generale) ---
    df_alloggi_target_area = df_alloggi_long[df_alloggi_long["Residenza"] == meno_forte_area]
    if not df_alloggi_target_area.empty and df_alloggi_target_area["Valore"].sum() > 0:
        distrib_alloggi_target_area_pct = (df_alloggi_target_area.groupby("Alloggio")["Valore"].sum() /
                                           df_alloggi_target_area["Valore"].sum() * 100).sort_values(ascending=False)
        alloggio_preferito_silvia = distrib_alloggi_target_area_pct.index[0]
        print(f"🏠 Alloggio preferito dai residenti di '{meno_forte_area}' (in generale): '{alloggio_preferito_silvia}' ({distrib_alloggi_target_area_pct.iloc[0]:.1f}%).")
        print("Per i residenti di quest'area, privilegia offerte di turismo interno che includano questo tipo di alloggio.")

        if len(distrib_alloggi_target_area_pct) > 1:
            alloggio_meno_usato_silvia = distrib_alloggi_target_area_pct.index[-1]
            print(f"Alternative: L'alloggio meno usato in '{meno_forte_area}' è '{alloggio_meno_usato_silvia}' ({distrib_alloggi_target_area_pct.iloc[-1]:.1f}%). Potrebbe esserci un'opportunità per sviluppare offerte innovative.")
    else:
        print(f"Nessun dato sugli alloggi per '{meno_forte_area}'.")

### Menù interattivo per il Bot
print("Benvenuto nel bot di Travel Target! 👋\n")
print("Seleziona chi sei:")
print("1. Giulia – Marketing") # Ho rimosso "Nord-ovest" qui per rendere la scelta più generica
print("2. Luca – Data analyst agenzia viaggi")
print("3. Silvia – Ministero del Turismo")

scelta = input("Inserisci il numero corrispondente: ")

if scelta == "1":
    print("\nSei Giulia. La tua area di interesse è solitamente il Nord-ovest.")
    conferma = input("Vuoi analizzare i dati per il Nord-ovest? (sì/no): ").lower()

    if conferma == "sì" or conferma == "si":
        bot_giulia("Nord-ovest")
    else:
        aree_disponibili = df_dest_long["Residenza"].unique().tolist()
        aree_disponibili.sort()
        print("\nSeleziona la tua macro-area di residenza di interesse:")
        for i, nome in enumerate(aree_disponibili):
            print(f"{i+1}. {nome}")

        while True: # Loop per assicurare una scelta valida
            try:
                s_area_idx = int(input("Numero area: ")) - 1
                if 0 <= s_area_idx < len(aree_disponibili):
                    area_scelta_giulia = aree_disponibili[s_area_idx]
                    bot_giulia(area_scelta_giulia)
                    break # Esci dal loop se la scelta è valida
                else:
                    print("Numero non valido. Riprova.")
            except ValueError:
                print("Input non valido. Inserisci un numero.")

elif scelta == "2":
    bot_luca()
elif scelta == "3":
    bot_silvia()
else:
    print("Scelta non valida. Riprova.")

Benvenuto nel bot di Travel Target! 👋

Seleziona chi sei:
1. Giulia – Marketing
2. Luca – Data analyst agenzia viaggi
3. Silvia – Ministero del Turismo
Inserisci il numero corrispondente: 1

Sei Giulia. La tua area di interesse è solitamente il Nord-ovest.
Vuoi analizzare i dati per il Nord-ovest? (sì/no): sì

🎯 Benvenuta Giulia! Area di riferimento selezionata: Nord-ovest
Obiettivo: promuovere mete italiane poco frequentate, raggiungibili in auto.

📊 Destinazioni italiane meno preferite (in ordine crescente):
- Sud, Isole: 8.5%
- Centro (IT): 16.4%
- Nord: 75.2%

👉 Suggerimento destinazioni: i residenti del Nord-ovest scelgono poco la destinazione italiana 'Sud, Isole'.
Potresti lanciare una campagna promozionale mirata verso questa zona.

🚗 Mezzo di trasporto più usato dai residenti del Nord-ovest: 'Auto' (64.9%).
Dato il tuo interesse per i viaggi in auto, in questa area l'auto è usata nel 64.9% dei casi.
Considera di promuovere viaggi verso destinazioni interne utilizzando principa